In [2]:
from qiskit.circuit.library import QAOAAnsatz
from qiskit.transpiler import CouplingMap
from qiskit.quantum_info import Pauli, SparsePauliOp, Operator, Statevector
from qiskit.result import QuasiDistribution

from qiskit_aer.primitives import Sampler as AerSampler


from qiskit_algorithms import QAOA, NumPyMinimumEigensolver
from qiskit_algorithms.optimizers import COBYLA
from qiskit_algorithms.utils import algorithm_globals


import numpy as np
import networkx as nx

def get_operator(G):
    r"""Generate Hamiltonian for the graph partitioning
    Notes:
        Goals:
            1 Separate the vertices into two set of the same size.
            2 Make sure the number of edges between the two set is minimized.
        Hamiltonian:
            H = H_A + H_B
            H_A = sum\_{(i,j)\in E}{(1-ZiZj)/2}
            H_B = (sum_{i}{Zi})^2 = sum_{i}{Zi^2}+sum_{i!=j}{ZiZj}
            H_A is for achieving goal 2 and H_B is for achieving goal 1.
    Args:
        G: Graph.
    Returns:
        Operator for the Hamiltonian
        A constant shift for the obj function.
    """
    weight_matrix = nx.adjacency_matrix(G).todense()

    num_nodes = len(weight_matrix)
    pauli_list = []
    coeffs = []
    shift = 0

    for i in range(num_nodes):
        for j in range(i):
            if weight_matrix[i, j] != 0:
                x_p = np.zeros(num_nodes, dtype=bool)
                z_p = np.zeros(num_nodes, dtype=bool)
                z_p[i] = True
                z_p[j] = True
                pauli_list.append(Pauli((z_p, x_p)))
                coeffs.append(-0.5)
                shift += 0.5

    for i in range(num_nodes):
        for j in range(num_nodes):
            if i != j:
                x_p = np.zeros(num_nodes, dtype=bool)
                z_p = np.zeros(num_nodes, dtype=bool)
                z_p[i] = True
                z_p[j] = True
                pauli_list.append(Pauli((z_p, x_p)))
                coeffs.append(1.0)
            else:
                shift += 1

    return SparsePauliOp(pauli_list, coeffs=coeffs), shift

def close_objective_value(x, w):
    """Compute the value of a cut, according to the "closest" binary string i.e. the state with the highest probability.
    Args:
        x: Binary string as numpy array.
        w: Adjacency matrix.
    Returns:
        Value of the cut.
    """
    X = np.outer(x, (1 - x))
    w_01 = np.where(w != 0, 1, 0)
    return np.sum(w_01 * X)

def bitfield(n, L):
    result = np.binary_repr(n, L)
    return [int(digit) for digit in result]  # [2:] to chop off the "0b" part

def sample_most_likely(state_vector, n):
    """Compute the most likely binary string from state vector.
    Args:
        state_vector: State vector or quasi-distribution.
        n: Number of nodes

    Returns:
        Binary string as an array of ints.
    """
    if isinstance(state_vector, QuasiDistribution):
        values = state_vector.binary_probabilities(n)
        k = max(values, key=values.get)
        return np.asarray([int(i) for i in k])
    elif isinstance(state_vector, Statevector):
        values = state_vector
        k = np.argmax(np.abs(values))
        x = bitfield(k, n)
        x.reverse()
        return np.asarray(x)
    else:
        raise Exception("Error")



def QAOA_circuit(G, shots=10000, reps=2, method ='statevector', device=None):

    n = G.number_of_nodes()
    w = nx.adjacency_matrix(G).todense()


    qubit_op, offset = get_operator(G)
    cm = CouplingMap().from_full(n)

    sampler = AerSampler(backend_options={"method": method, "device": device,},
                         transpile_options={"coupling_map": cm},
                         run_options={"shots": shots})
    
    circuit = QAOAAnsatz(cost_operator=qubit_op, reps=reps)
    circuit.measure_all(inplace=True, add_bits=True)

    num_parameters = circuit.num_parameters
    result = sampler.run(circuit, num_parameters*[np.pi/2]).result()
    
    quasidist = result.quasi_dists[0]
    x = sample_most_likely(quasidist, n)

    return quasidist, x, close_objective_value(x, w), offset


def QAOA_experiment(G, seed=10, shots=10000, reps=2, method ='statevector', optimizer=None, device=None):

    n = G.number_of_nodes()
    w = nx.adjacency_matrix(G).todense()

    if optimizer is None:
        optimizer = COBYLA()

    algorithm_globals.random_seed = seed

    qubit_op, offset = get_operator(G)
    cm = CouplingMap().from_full(n)
    sampler = AerSampler(backend_options={"method": method, "device": device,},
                         transpile_options={"coupling_map": cm},
                         run_options={"shots": shots})
    
    
    qaoa = QAOA(sampler, optimizer, reps=reps)
    result = qaoa.compute_minimum_eigenvalue(qubit_op)
    eigenvalue = result.eigenvalue

    x = sample_most_likely(result.eigenstate, n)
    
    return x, close_objective_value(x, w), eigenvalue, offset

####################################################################################################

# Needs to install networkx and qiskit_algorithms

# The number of nodes corresponds to the number of qubits (use an even number to avoid problems)
# Test both QAOA_citcuit and QAOA_algorithm


nod = 4                                            # Modify this for more/less qubits                     
deg = nod//2
G = nx.random_regular_graph(deg, nod, seed = 666)

print(QAOA_circuit(G, method ='statevector', device=None))
#print(QAOA_experiment(G, method ='statevector', device=None))

(array([0, 1, 0, 1]), 2, -3.7572000000000005, 6.0)
